In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 화면 없이 실행하고 싶으면 주석 해제
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)


In [15]:
from selenium.webdriver.common.by import By

# 1) 페이지 열기
driver.get("https://en.wikipedia.org/wiki/List_of_national_capitals_by_population")

# 2) 표(table)에서 첫 번째 5개 행(row)만 선택
rows = driver.find_elements(By.CSS_SELECTOR, "table.wikitable tbody tr")[1:6]

capitals = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    capital = cells[1].text             # 2번째 칸: 수도 이름
    country = cells[2].text             # 3번째 칸: 나라 이름
    capitals.append((capital, country))

driver.quit()  # 끝나면 브라우저 닫기
print("가져온 수도:", capitals)


가져온 수도: [('Tokyo', '14,094,034'), ('Moscow', '13,104,177'), ('Kinshasa', '12,691,000'), ('Jakarta', '10,562,088'), ('Lima', '10,151,000')]


In [16]:
from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent="capital_mapper")

locations = []
for city, country in capitals:
    # “수도, 나라” 형태로 검색
    place = geocoder.geocode(f"{city}, {country}")
    if place:
        locations.append({
            "name": city,
            "country": country,
            "lat": place.latitude,
            "lon": place.longitude
        })
    else:
        print(f"⚠️ {city} 좌표를 못 찾았어요.")

print("위경도 변환 결과:", locations)


⚠️ Moscow 좌표를 못 찾았어요.
⚠️ Kinshasa 좌표를 못 찾았어요.
⚠️ Lima 좌표를 못 찾았어요.
위경도 변환 결과: [{'name': 'Tokyo', 'country': '14,094,034', 'lat': 35.5843267, 'lon': 139.3501652}, {'name': 'Jakarta', 'country': '10,562,088', 'lat': -6.2885599, 'lon': 106.7781032}]


In [17]:
import folium

# 1) 지도를 만든 뒤 (전 세계가 보이도록 중심을 [20,0], 줌 레벨 2로 설정)
m = folium.Map(location=[20, 0], zoom_start=2)

# 2) 각 수도마다 마커(marker) 추가
for loc in locations:
    folium.Marker(
        location=[loc["lat"], loc["lon"]],
        popup=f"{loc['name']} ({loc['country']})"
    ).add_to(m)

# 3) HTML 파일로 저장
m.save("capitals_map.html")
print("✅ 지도가 'capitals_map.html'에 저장되었어요.")


✅ 지도가 'capitals_map.html'에 저장되었어요.
